# Pandas Groupby的一些延伸与练习

In [1]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                           'foo', 'bar', 'foo', 'foo'],
                   'B' : ['one', 'one', 'two', 'three',
                          'two', 'two', 'one', 'three'],
                   'C' : np.random.randn(8),
                   'D' : np.random.randn(8)})
df

,A,B,C,D
0,foo,one,-0.096804,0.907595
1,bar,one,-0.120722,0.069596
2,foo,two,2.154428,1.538896
3,bar,three,0.834632,-0.239432
4,foo,two,-0.008928,-0.338258
5,bar,two,1.469774,0.042729
6,foo,one,-0.089727,1.050386
7,foo,three,1.574685,-0.221205


### 统计出现次数

In [3]:
grouped = df.groupby('A')
grouped.count()

,B,C,D
A,,,
bar,3,3,3
foo,5,5,5


### 以两个为键

In [4]:
grouped = df.groupby(['A','B'])
grouped.count()

C  D
A   B          
bar one    1  1
    three  1  1
    two    1  1
foo one    2  2
    three  1  1
    two    2  2

### 自定义函数

In [7]:
def get_letter_type(letter):
    if letter.lower() in 'aeiou':
        return 'a'
    else:
        return 'b'
grouped = df.groupby(get_letter_type,axis = 1)
grouped.count()

,a,b
0,1,3
1,1,3
2,1,3
3,1,3
4,1,3
5,1,3
6,1,3
7,1,3


### 有重复值的时候进行的操作
* .groupby(level = 0)

level = 0指的是第一个索引，如果有多个索引，level可以为1，2 ....

In [8]:
s = pd.Series([1,2,3,1,2,3],[8,7,5,8,7,5])
s

8    1
7    2
5    3
8    1
7    2
5    3
dtype: int64

In [12]:
grouped = s.groupby(level = 0) #默认会排序，如果不想排序这可以加 sort = False

In [13]:
grouped.first()

5    3
7    2
8    1
dtype: int64

In [15]:
grouped.last()

5    3
7    2
8    1
dtype: int64

In [17]:
grouped.sum()

5    6
7    4
8    2
dtype: int64

* get_group(),指定关注对象

In [18]:
df2 = pd.DataFrame({'X':['A','B','A','B'],'Y':[1,2,3,4]})
df2

,X,Y
0,A,1
1,B,2
2,A,3
3,B,4


In [22]:
df2.groupby(['X']).get_group('A') #关注A

,X,Y
0,A,1
2,A,3


In [20]:
df2.groupby(['X']).get_group('B') #关注B

,X,Y
1,B,2
3,B,4


### 多重索引
* pd.MultiIndex.from_arrays(arrays ,names = )
* .groupby(level = )

In [23]:
arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
          ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]

In [26]:
index = pd.MultiIndex.from_arrays(arrays,names = ['first','second'])
index

MultiIndex(levels=[['bar', 'baz', 'foo', 'qux'], ['one', 'two']],
           labels=[[0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 0, 1, 0, 1, 0, 1]],
           names=['first', 'second'])

In [27]:
s = pd.Series(np.random.randn(8),index = index)
s

first  second
bar    one       0.897028
       two      -0.931645
baz    one       0.976629
       two       0.856479
foo    one       0.912936
       two       0.103939
qux    one       1.652940
       two       1.358233
dtype: float64

In [31]:
grouped = s.groupby(level = 0)  #按照第一个索引groupby
grouped.sum()

first
bar   -0.034617
baz    1.833107
foo    1.016875
qux    3.011173
dtype: float64

In [32]:
grouped = s.groupby(level = 1)   #按照第二个索引groupby
grouped.sum()

second
one    4.439533
two    1.387005
dtype: float64

In [34]:
grouped = s.groupby(level = 'first')  #也可以指定索引名字
grouped.sum()

first
bar   -0.034617
baz    1.833107
foo    1.016875
qux    3.011173
dtype: float64

### 其它一些特定的操作
* .aggregate()

In [35]:
df

,A,B,C,D
0,foo,one,-0.096804,0.907595
1,bar,one,-0.120722,0.069596
2,foo,two,2.154428,1.538896
3,bar,three,0.834632,-0.239432
4,foo,two,-0.008928,-0.338258
5,bar,two,1.469774,0.042729
6,foo,one,-0.089727,1.050386
7,foo,three,1.574685,-0.221205


In [36]:
grouped = df.groupby(['A','B'])
grouped.aggregate(np.sum)

C         D
A   B                        
bar one   -0.120722  0.069596
    three  0.834632 -0.239432
    two    1.469774  0.042729
foo one   -0.186531  1.957981
    three  1.574685 -0.221205
    two    2.145500  1.200639

### groupby后不改变index的形状
* 指定as_index =False
* .reset_index()

In [37]:
#可以指定as_index = False
grouped = df.groupby(['A','B'],as_index = False)
grouped.aggregate(np.sum)

,A,B,C,D
0,bar,one,-0.120722,0.069596
1,bar,three,0.834632,-0.239432
2,bar,two,1.469774,0.042729
3,foo,one,-0.186531,1.957981
4,foo,three,1.574685,-0.221205
5,foo,two,2.145500,1.200639


In [38]:
#也可以用.reset_index()实现和上面一样的操作
df.groupby(['A','B']).sum().reset_index()

,A,B,C,D
0,bar,one,-0.120722,0.069596
1,bar,three,0.834632,-0.239432
2,bar,two,1.469774,0.042729
3,foo,one,-0.186531,1.957981
4,foo,three,1.574685,-0.221205
5,foo,two,2.145500,1.200639


### .size显示分完类后(组合)的个数

In [39]:
grouped = df.groupby(['A','B'])
grouped.size()

A    B    
bar  one      1
     three    1
     two      1
foo  one      2
     three    1
     two      2
dtype: int64

### 显示统计特性

In [41]:
grouped.describe()

C                                                              \
          count      mean       std       min       25%       50%       75%   
A   B                                                                         
bar one     1.0 -0.120722       NaN -0.120722 -0.120722 -0.120722 -0.120722   
    three   1.0  0.834632       NaN  0.834632  0.834632  0.834632  0.834632   
    two     1.0  1.469774       NaN  1.469774  1.469774  1.469774  1.469774   
foo one     2.0 -0.093266  0.005004 -0.096804 -0.095035 -0.093266 -0.091496   
    three   1.0  1.574685       NaN  1.574685  1.574685  1.574685  1.574685   
    two     2.0  1.072750  1.529724 -0.008928  0.531911  1.072750  1.613589   

                        D                                                    \
                max count      mean       std       min       25%       50%   
A   B                                                                         
bar one   -0.120722   1.0  0.069596       NaN  0.069596  0.069596  0.069596   
    three  0.834632   1.0 -0.239432       NaN -0.239432 -0.239432 -0.239432   
    two    1.469774   1.0  0.042729       NaN  0.042729  0.042729  0.042729   
foo one   -0.089727   2.0  0.978990  0.100968  0.907595  0.943293  0.978990   
    three  1.574685   1.0 -0.221205       NaN -0.221205 -0.221205 -0.221205   
    two    2.154428   2.0  0.600319  1.327348 -0.338258  0.131031  0.600319   

                               
                75%       max  
A   B                          
bar one    0.069596  0.069596  
    three -0.239432 -0.239432  
    two    0.042729  0.042729  
foo one    1.014688  1.050386  
    three -0.221205 -0.221205  
    two    1.069608  1.538896

### 统计多个特性

In [42]:
grouped = df.groupby('A')
grouped['C'].agg([np.sum,np.mean,np.std])

,sum,mean,std
A,,,
bar,2.183684,0.727895,0.800603
foo,3.533653,0.706731,1.077190


In [44]:
# 指定统计指标的列名
grouped['C'].agg({'res_sum':np.sum,'res_mean':np.mean,'res_std':np.std})

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


,res_mean,res_sum,res_std
A,,,
bar,0.727895,2.183684,0.800603
foo,0.706731,3.533653,1.077190
